# Par-Rate Bumps for Libor Curves
Investigate the impact of a par rate bump (bump to a curve benchmark used in curve building) on the curve

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
from timeit import default_timer as timer

In [ ]:
from financepy.utils import *
from financepy.products.rates import *

In [ ]:
valuation_date = Date(6, 10, 2001)
cal = CalendarTypes.UNITED_KINGDOM

### Instruments

In [ ]:
depo_dcc_type = DayCountTypes.ACT_360
depos = []
spot_days = 2
settle_dt = valuation_date.add_weekdays(spot_days)
depo = IborDeposit(settle_dt, "3M", 4.2/100.0, depo_dcc_type, cal_type=cal)
depos.append(depo)

fraDCCType = DayCountTypes.ACT_360
fras = []
fra = IborFRA(settle_dt.add_tenor("3M"), "3M", 4.20/100.0, fraDCCType, cal_type=cal)
fras.append(fra)

swaps = []
swapType = SwapTypes.PAY
fixedDCCType = DayCountTypes.THIRTY_E_360_ISDA
fixed_freqType = FrequencyTypes.SEMI_ANNUAL

swap = IborSwap(settle_dt, "1Y", swapType, 4.20/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settle_dt, "2Y", swapType, 4.30/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settle_dt, "3Y", swapType, 4.70/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settle_dt, "5Y", swapType, 5.40/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settle_dt, "7Y", swapType, 5.70/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settle_dt, "10Y", swapType, 6.00/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settle_dt, "12Y", swapType, 6.10/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settle_dt, "15Y", swapType, 5.90/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settle_dt, "20Y", swapType, 5.60/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settle_dt, "25Y", swapType, 5.55/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)

Build curves using different interpolations

In [ ]:
check_refit_flag = True
iborCurves = {}
iborCurveShockers = {}
optional_interp_params = {'sigma' : 0.01} # only relevant for interp_type == InterpTypes.TENSION_ZERO_RATES

interp_types_to_use = {
    'FF':(InterpTypes.FLAT_FWD_RATES,{}),
    'LF':(InterpTypes.LINEAR_FWD_RATES,{}),
    'LZ':(InterpTypes.LINEAR_ZERO_RATES,{}),
    'CZ':(InterpTypes.NATCUBIC_ZERO_RATES,{}),
    'TZ1':(InterpTypes.TENSION_ZERO_RATES,{'sigma' : 0.1}),
    'TZ2':(InterpTypes.TENSION_ZERO_RATES,{'sigma' : 2.0}),
    'TZ3':(InterpTypes.TENSION_ZERO_RATES,{'sigma' : 10.0}),
}

for key, interp_type in interp_types_to_use.items():
    print('Processing:', key, interp_type[0])
    ibor_curve = IborSingleCurve(valuation_date, depos, fras, swaps, interp_type[0], check_refit_flag=check_refit_flag, **interp_type[1])
    iborCurves[key] = ibor_curve
    curveShocker = IborSingleCurveParShocker(ibor_curve)
    iborCurveShockers[key] = curveShocker


### Look at the benchmark report for one of the curves
(they all have the same benchmarks)


In [ ]:
# Print DataFrames in full
pd.set_option('display.max_rows', None,
                    'display.max_columns', None,
                    'display.float_format', lambda x:f'{x:.4f}'
                    )

In [ ]:
# examine the last fit_report
df_br = curveShocker.benchmarks_report()
df_br

### Choose the benchmark to bump and apply the bump

In [ ]:
par_rate_bump = 1*0.0001
benchmark_idx =7
print(df_br.loc[benchmark_idx])
bumped_curves = {}

for key, shocker in iborCurveShockers.items():
    print('Processing:', key)
    bumped_curve = shocker.apply_bump_to_benchmark(benchmark_idx, par_rate_bump)
    bumped_curves[key] = bumped_curve

### Curve Shape Analysis

Check instantaneous (ON) forward rates and see how they are affected by the par rate bump
We are looking how 'local' are the changes in fwd rates -- the further they extend beyond benchmarks
near the bumped one, the less local the bump and, generally, less financially justifyable the interpolation scheme

In [ ]:
# %matplotlib widget  # keep only if you're in Jupyter

years = np.linspace(1.0/365.0, 25.0, int(25*365), endpoint=True)
plotDates = [settle_dt.add_years(float(y)) for y in years]  # list of Dates

benchmark_years = (df_br['maturity_dt'] - valuation_date) / G_DAYS_IN_YEARS
# ensure it's a 1D numpy array for plotting
benchmark_years = np.asarray(benchmark_years, dtype=float)

keys_to_plot_collections = [['FF', 'LF', 'LZ'], ['FF', 'LZ', 'CZ'], ['CZ', 'TZ1', 'TZ2', 'TZ3']]

for keys_to_plot in keys_to_plot_collections:
    plt.figure(figsize=(8, 5))

    for key in keys_to_plot:
        base_curve = iborCurves[key]
        bumped_curve = bumped_curves[key]

        base_fwd_rates   = base_curve.fwd(plotDates)    # expects list[Date]
        bumped_fwd_rates = bumped_curve.fwd(plotDates)

        # convert to numpy just in case theyÃƒÆ’Ã‚Â¢ÃƒÂ¢Ã¢â‚¬Å¡Ã‚Â¬ÃƒÂ¢Ã¢â‚¬Å¾Ã‚Â¢re lists
        base_fwd_rates   = np.asarray(base_fwd_rates, dtype=float)
        bumped_fwd_rates = np.asarray(bumped_fwd_rates, dtype=float)

        bump_bps = (bumped_fwd_rates - base_fwd_rates) * 10000.0
        label_interp = getattr(interp_types_to_use[key][0], "name", str(interp_types_to_use[key][0]))
        plt.plot(years, bump_bps, label=f"interp={label_interp}")

    # show benchmark maturities at zero bump
    plt.scatter(benchmark_years, np.zeros_like(benchmark_years), marker='o', label='benchmark maturity dates')

    plt.title(f'Par rate bump to benchmark {benchmark_idx}')
    plt.xlabel("Years")
    plt.ylabel("Bump (bps)")
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()
